In [40]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [41]:
# To decide churn metric as LOGO or ACV

df = pd.read_csv("../data/dataset1.csv")  

SEG_COL = "sales_segment"   # try: "sales_segment", "product_tier", "company_size_bucket", "region", "acquisition_channel", "is_eu", "industry"

ID_COL    = "customer_id"
CHURN_COL = "is_churned"              
ACV_COL   = "annual_contract_value"   

df[SEG_COL] = df[SEG_COL].fillna("Unknown")
df[CHURN_COL] = pd.to_numeric(df[CHURN_COL], errors="coerce").fillna(0).astype(int)
df[ACV_COL] = pd.to_numeric(df[ACV_COL], errors="coerce").fillna(0.0)

churned = df[df[CHURN_COL] == 1].copy()
total_churned_logos = churned[ID_COL].nunique()
total_churned_acv   = churned[ACV_COL].sum()

summary = (
    df.groupby(SEG_COL, dropna=False)
      .agg(customers=(ID_COL, "nunique"),
           churned_customers=(CHURN_COL, "sum"),
           total_acv=(ACV_COL, "sum"))
      .reset_index()
)

churn_by_seg = (
    churned.groupby(SEG_COL, dropna=False)
           .agg(churned_acv=(ACV_COL, "sum"))
           .reset_index()
)

summary = summary.merge(churn_by_seg, on=SEG_COL, how="left").fillna({"churned_acv": 0.0})

summary["logo_churn_rate"] = summary["churned_customers"] / summary["customers"]
summary["acv_churn_rate"]  = summary["churned_acv"] / summary["total_acv"].replace({0: pd.NA})

summary["logo_churn_share"] = summary["churned_customers"] / (total_churned_logos if total_churned_logos else 1)
summary["acv_churn_share"]  = summary["churned_acv"] / (total_churned_acv if total_churned_acv else 1)

logo_top = summary.sort_values("logo_churn_share", ascending=False).head(10)
acv_top  = summary.sort_values("acv_churn_share",  ascending=False).head(10)

print("Top by LOGO churn share:")
print(logo_top[[SEG_COL, "customers", "churned_customers", "logo_churn_share", "logo_churn_rate"]].to_string(index=False))

print("\nTop by ACV churn share:")
print(acv_top[[SEG_COL, "total_acv", "churned_acv", "acv_churn_share", "acv_churn_rate"]].to_string(index=False))

top_logo_seg = logo_top.iloc[0][SEG_COL] if len(logo_top) else None
top_acv_seg  = acv_top.iloc[0][SEG_COL]  if len(acv_top)  else None
print("\nDecision helper:")
print("Top logo segment:", top_logo_seg)
print("Top ACV segment :", top_acv_seg)


Top by LOGO churn share:
sales_segment  customers  churned_customers  logo_churn_share  logo_churn_rate
   SMB_Inside       1320                217          0.596154         0.164394
    SMB_Field        574                 97          0.266484         0.168990
    MidMarket        632                 29          0.079670         0.045886
   Enterprise        474                 21          0.057692         0.044304

Top by ACV churn share:
sales_segment   total_acv  churned_acv  acv_churn_share  acv_churn_rate
   Enterprise 84265245.37   3666669.15         0.701746        0.043513
    MidMarket 15870773.45    743722.20         0.142337        0.046861
   SMB_Inside  5436775.50    595852.90         0.114037        0.109597
    SMB_Field  2260570.77    218822.49         0.041879        0.096800

Decision helper:
Top logo segment: SMB_Inside
Top ACV segment : Enterprise


In [42]:
# Dataset 1 cleaning and preprocessing
df1 = pd.read_csv("../data/dataset1.csv") 

# Checking missingness
missing_counts = df1.isna().sum().sort_values(ascending=False)
missing_rates = (df1.isna().mean()*100).round(2).sort_values(ascending=False)

print("Missing counts (top):")
print(missing_counts.head(5))

''' Missing counts (top):
contract_end_date           1428
industry                     862 '''

df1.loc[df1["industry"].isna(), ["customer_id","company_name","country","region","industry"]].head(20)

df1["industry_is_missing"] = df1["industry"].isna()
df1["industry"] = df1["industry"].fillna("Unknown")

print("\nIndustry missing after fix:", df1["industry"].isna().sum())
print(df1["industry"].value_counts().head(10))

Missing counts (top):
contract_end_date           1428
industry                     862
customer_id                    0
sales_segment                  0
initial_onboarding_score       0
dtype: int64

Industry missing after fix: 0
industry
Unknown                  862
Ecommerce                278
Hospitality              275
Healthcare               273
Logistics                268
Professional Services    264
Wholesale                261
Manufacturing            261
Retail                   258
Name: count, dtype: int64


In [43]:
# fixing company_size_bucket: common Excel auto-date formatting issues in this file 
print(df1["company_size_bucket"].value_counts())

SIZE_BUCKET_FIX = {"10-Jan": "1-10", "Nov-50": "11-50"}
VALID_BUCKETS = {"1-10","11-50","51-200","201-1000","1000+"}

df1["company_size_bucket_original"] = df1["company_size_bucket"]
df1["company_size_bucket"] = df1["company_size_bucket"].replace(SIZE_BUCKET_FIX)

df1["company_size_bucket_was_fixed"] = (
    df1["company_size_bucket_original"].notna()
    & (df1["company_size_bucket_original"] != df1["company_size_bucket"])
)

print("\nFixed rows:", int(df1["company_size_bucket_was_fixed"].sum()))
print(df1["company_size_bucket"].value_counts())


company_size_bucket
10-Jan      1035
Nov-50       859
51-200       632
201-1000     307
1000+        167
Name: count, dtype: int64

Fixed rows: 1894
company_size_bucket
1-10        1035
11-50        859
51-200       632
201-1000     307
1000+        167
Name: count, dtype: int64


In [44]:
# ---- A1) Parse dates
df1["contract_start_dt"] = pd.to_datetime(df1["contract_start_date"], errors="coerce")
df1["contract_end_dt"]   = pd.to_datetime(df1["contract_end_date"], errors="coerce")

print("Missing start dates:", df1["contract_start_dt"].isna().sum())
print("Missing end dates:", df1["contract_end_dt"].isna().sum())

df1["end_before_start"] = (
    df1["contract_start_dt"].notna()
    & df1["contract_end_dt"].notna()
    & (df1["contract_end_dt"] < df1["contract_start_dt"])
)
print("End before start:", int(df1["end_before_start"].sum()))

df1["has_missing_contract_end_date"] = df1["contract_end_dt"].isna()

print(pd.crosstab(df1["renewed_flag"], df1["has_missing_contract_end_date"]))
print(pd.crosstab(df1["is_churned"], df1["has_missing_contract_end_date"]))

# Contract term in days (only where end exists)
df1["contract_term_days"] = (df1["contract_end_dt"] - df1["contract_start_dt"]).dt.days

print("\nContract term (days) stats:")
print(df1["contract_term_days"].describe())

print("\nMost common terms:", df1["contract_term_days"].value_counts().head(10))

df1["violation_churn_missing_end_date"] = (df1["is_churned"] == 1) & df1["contract_end_dt"].isna()

# If churned but term >= ~12 months, that contradicts "early termination"
df1["violation_churn_not_early_term"] = (
    (df1["is_churned"] == 1)
    & df1["contract_term_days"].notna()
    & (df1["contract_term_days"] >= 365)
)

print("Churned but missing end date:", int(df1["violation_churn_missing_end_date"].sum()))
print("Churned but not early termination:", int(df1["violation_churn_not_early_term"].sum()))

# view the rows if any violations exist
df1.loc[df1["violation_churn_not_early_term"],
        ["customer_id","contract_start_date","contract_end_date","contract_term_days","renewed_flag","is_churned"]].head(20)

# print violation counts with renewed_flag==0 and is_churned==1
print("Violation counts with renewed_flag==0 and is_churned==1:",   
      int(df1[(df1["violation_churn_not_early_term"]) & (df1["renewed_flag"] == 0) & (df1["is_churned"] == 1)].shape[0])
     )

Missing start dates: 0
Missing end dates: 1428
End before start: 0
has_missing_contract_end_date  False  True 
renewed_flag                               
0                               1572      0
1                                  0   1428
has_missing_contract_end_date  False  True 
is_churned                                 
0                               1208   1428
1                                364      0

Contract term (days) stats:
count    1572.000000
mean      299.603053
std        99.361937
min        91.000000
25%       183.000000
50%       366.000000
75%       366.000000
max       366.000000
Name: contract_term_days, dtype: float64

Most common terms: contract_term_days
366.0    1068
183.0     372
91.0      132
Name: count, dtype: int64
Churned but missing end date: 0
Churned but not early termination: 69
Violation counts with renewed_flag==0 and is_churned==1: 69


In [45]:
valid_channels = {"Inbound","Outbound","Partner","SelfServe"}
df1["acquisition_channel_invalid"] = df1["acquisition_channel"].notna() & ~df1["acquisition_channel"].isin(valid_channels)
print("Invalid acquisition_channel:", int(df1["acquisition_channel_invalid"].sum()))
df1.loc[df1["acquisition_channel_invalid"], ["customer_id","acquisition_channel"]].head(10)

# EU consistency check
df1["eu_region_mismatch"] = (
    (df1["region"].notna()) &
    (((df1["is_eu"] == 1) & (df1["region"] != "Europe")) |
     ((df1["is_eu"] == 0) & (df1["region"] == "Europe")))
)
print("EU-region mismatches:", int(df1["eu_region_mismatch"].sum()))

df1["acv_invalid"] = df1["annual_contract_value"].notna() & (df1["annual_contract_value"] <= 0)
df1["discount_invalid"] = df1["discount_pct"].notna() & ~df1["discount_pct"].between(0, 1)
df1["onboarding_invalid"] = df1["initial_onboarding_score"].notna() & ~df1["initial_onboarding_score"].between(0, 10)

print("ACV invalid:", int(df1["acv_invalid"].sum()))
print("Discount invalid:", int(df1["discount_invalid"].sum()))
print("Onboarding invalid:", int(df1["onboarding_invalid"].sum()))

print("Duplicate customer_id:", int(df1["customer_id"].duplicated().sum()))


Invalid acquisition_channel: 0
EU-region mismatches: 0
ACV invalid: 0
Discount invalid: 0
Onboarding invalid: 0
Duplicate customer_id: 0


In [46]:
df1.head()
df1.columns
cols_to_drop = [
    "company_size_bucket_original",
    "company_size_bucket_was_fixed",
    "end_before_start",
    "violation_churn_missing_end_date",
    "acquisition_channel_invalid",
    "eu_region_mismatch",
    "acv_invalid",
    "discount_invalid",
    "onboarding_invalid",
]

# only drop columns that actually exist (safe)
df1.drop(columns=[c for c in cols_to_drop if c in df1.columns], inplace=True)

df1.columns

Index(['customer_id', 'company_name', 'country', 'region', 'is_eu', 'industry',
       'company_size_bucket', 'annual_contract_value', 'product_tier',
       'sales_segment', 'acquisition_channel', 'contract_start_date',
       'contract_end_date', 'renewed_flag', 'discount_pct',
       'initial_onboarding_score', 'is_churned', 'industry_is_missing',
       'contract_start_dt', 'contract_end_dt', 'has_missing_contract_end_date',
       'contract_term_days', 'violation_churn_not_early_term'],
      dtype='object')

In [47]:
ID_COL = "customer_id"
CHURN_COL = "is_churned"
ACV_COL = "annual_contract_value"

FEATURES = ["sales_segment", "company_size_bucket", "product_tier", "acquisition_channel"]

acv_metrics = {}

for feat in FEATURES:
    grouped = df1.groupby(feat, dropna=False)
    
    total_acv = grouped[ACV_COL].sum()
    churned_acv = grouped.apply(lambda x: x.loc[x[CHURN_COL]==1, ACV_COL].sum())
    
    acv_churn_rate = churned_acv / total_acv.replace({0: pd.NA})
    total_churned_acv = churned_acv.sum()
    acv_churn_share = churned_acv / (total_churned_acv if total_churned_acv else 1)
    
    acv_metrics[feat] = pd.DataFrame({
        "total_acv": total_acv,
        "churned_acv": churned_acv,
        "acv_churn_rate": acv_churn_rate,
        "acv_churn_share": acv_churn_share
    }).sort_values("acv_churn_share", ascending=False)

# Example: ACV churn by sales segment
print(acv_metrics["sales_segment"])

                 total_acv  churned_acv  acv_churn_rate  acv_churn_share
sales_segment                                                           
Enterprise     84265245.37   3666669.15        0.043513         0.701746
MidMarket      15870773.45    743722.20        0.046861         0.142337
SMB_Inside      5436775.50    595852.90        0.109597         0.114037
SMB_Field       2260570.77    218822.49        0.096800         0.041879


/var/folders/_f/ttmxczgj72j0yw73z__5fnyw0000gn/T/ipykernel_36375/2489933428.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  churned_acv = grouped.apply(lambda x: x.loc[x[CHURN_COL]==1, ACV_COL].sum())
/var/folders/_f/ttmxczgj72j0yw73z__5fnyw0000gn/T/ipykernel_36375/2489933428.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  churned_acv = grouped.apply(lambda x: x.loc[x[CHURN_COL]==1, ACV_COL].sum())
/var/fol

In [48]:
# Categorize tenure roughly
bins = [0, 90, 180, 365, 9999]
labels = ["0-3mo", "3-6mo", "6-12mo", "12+mo"]
df1["tenure_bucket"] = pd.cut(df1["contract_term_days"].fillna(0), bins=bins, labels=labels)

tenure_acv = df1.groupby("tenure_bucket")[ACV_COL].apply(lambda x: x[df1[CHURN_COL]==1].sum())
print(tenure_acv)

discount_acv = df1.groupby(df1["discount_pct"]>0)[ACV_COL].apply(lambda x: x[df1[CHURN_COL]==1].sum())
print(discount_acv)

tenure_bucket
0-3mo           0.00
3-6mo      257288.43
6-12mo    2713792.09
12+mo     2253986.22
Name: annual_contract_value, dtype: float64
discount_pct
False          0.00
True     5225066.74
Name: annual_contract_value, dtype: float64


/var/folders/_f/ttmxczgj72j0yw73z__5fnyw0000gn/T/ipykernel_36375/482229129.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tenure_acv = df1.groupby("tenure_bucket")[ACV_COL].apply(lambda x: x[df1[CHURN_COL]==1].sum())


In [49]:
# Weighted average onboarding score of churned vs retained
weighted_onboarding_churned = (df1.loc[df1[CHURN_COL]==1, "initial_onboarding_score"] * df1.loc[df1[CHURN_COL]==1, ACV_COL]).sum() / df1.loc[df1[CHURN_COL]==1, ACV_COL].sum()
weighted_onboarding_retained = (df1.loc[df1[CHURN_COL]==0, "initial_onboarding_score"] * df1.loc[df1[CHURN_COL]==0, ACV_COL]).sum() / df1.loc[df1[CHURN_COL]==0, ACV_COL].sum()

print("Weighted onboarding (churned):", weighted_onboarding_churned)
print("Weighted onboarding (retained):", weighted_onboarding_retained)

Weighted onboarding (churned): 6.041789846114003
Weighted onboarding (retained): 6.356868688437809


In [50]:
# ===
#                total_acv         churned_acv     acv_churn_rate   acv_churn_share
#sales_segment                                                           

#Enterprise     84,265,245.37   3,666,669.15       0.043513         0.701746
#MidMarket      15,870,773.45    743,722.20        0.046861         0.142337
#SMB_Inside      5,436,775.50    595,852.90        0.109597         0.114037
#SMB_Field       2,260,570.77    218,822.49        0.096800         0.041879

##tenure_bucket
#0-3mo           0.00
#3-6mo      257,288.43
#6-12mo    2,713,792.09
#12+mo     2,253,986.22

#discount_pct
#False          0.00
#True     5,225,066.74

#Weighted onboarding (churned): 6.041789846114003
#Weighted onboarding (retained): 6.356868688437809
# ===

In [51]:
df1

,customer_id,company_name,country,region,is_eu,industry,company_size_bucket,annual_contract_value,product_tier,sales_segment,...,discount_pct,initial_onboarding_score,is_churned,industry_is_missing,contract_start_dt,contract_end_dt,has_missing_contract_end_date,contract_term_days,violation_churn_not_early_term,tenure_bucket
0,C100000,Company_0,Canada,North America,0,Healthcare,51-200,12999.25,Growth,MidMarket,...,0.03,7.0,0,False,2023-05-02,NaT,True,NaN,False,NaN
1,C100001,Company_1,UK,Europe,1,Wholesale,1-10,1799.72,Starter,SMB_Field,...,0.15,4.9,0,False,2023-02-18,NaT,True,NaN,False,NaN
2,C100002,Company_2,US,North America,0,Manufacturing,1-10,1770.83,Starter,SMB_Field,...,0.37,4.8,1,False,2023-11-12,2024-05-13,False,183.0,False,6-12mo
3,C100003,Company_3,France,Europe,1,Unknown,1-10,1790.30,Starter,SMB_Field,...,0.10,8.0,0,True,2023-08-06,NaT,True,NaN,False,NaN
4,C100004,Company_4,Netherlands,Europe,1,Professional Services,1-10,1552.00,Starter,SMB_Inside,...,0.11,3.8,1,False,2023-09-09,2024-03-10,False,183.0,False,6-12mo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,C102995,Company_2995,Canada,North America,0,Healthcare,11-50,7347.89,Growth,SMB_Inside,...,0.12,8.6,0,False,2023-03-10,NaT,True,NaN,False,NaN
2996,C102996,Company_2996,UK,Europe,1,Hospitality,1000+,381926.86,Enterprise,Enterprise,...,0.15,7.0,0,False,2023-05-24,NaT,True,NaN,False,NaN
2997,C102997,Company_2997,UK,Europe,1,Unknown,201-1000,116504.64,Growth,Enterprise,...,0.06,7.2,0,True,2023-03-10,NaT,True,NaN,False,NaN
2998,C102998,Company_2998,Spain,Europe,1,Manufacturing,1000+,461427.14,Growth,Enterprise,...,0.18,4.8,0,False,2023-07-18,NaT,True,NaN,False,NaN


In [52]:
# Churned customers with a valid contract_end_dt
churned_known = df1[(df1["is_churned"] == 1) & df1["contract_end_dt"].notna()].copy()

# Churned customers with missing or ambiguous end date
churned_ambiguous = df1[(df1["is_churned"] == 1) & df1["contract_end_dt"].isna()].copy()

churned_known["churn_quarter"] = churned_known["contract_end_dt"].dt.to_period("Q")

acv_churn_by_quarter = (
    churned_known.groupby("churn_quarter")["annual_contract_value"]
    .sum()
    .sort_index()
)
print(acv_churn_by_quarter)

churn_quarter
2023Q2      58065.46
2023Q3    1348584.47
2023Q4     297539.70
2024Q1    1629477.74
2024Q2    1891399.37
Freq: Q-DEC, Name: annual_contract_value, dtype: float64


In [53]:
total_acv_ambiguous = churned_ambiguous["annual_contract_value"].sum()
print("ACV churn with unknown timing:", total_acv_ambiguous)

ACV churn with unknown timing: 0.0


In [54]:
q3 = pd.Period('2023Q3', freq='Q-DEC')
q3_acv_by_segment = (
    churned_known[churned_known["churn_quarter"] == q3]
    .groupby("sales_segment")["annual_contract_value"]
    .sum()
)
print(q3_acv_by_segment)

sales_segment
Enterprise    1060637.12
MidMarket      159451.70
SMB_Field       38890.28
SMB_Inside      89605.37
Name: annual_contract_value, dtype: float64


# LOGOS

In [55]:
churn_by_segment = df1.groupby('sales_segment')['is_churned'].mean()
churn_by_size    = df1.groupby('company_size_bucket')['is_churned'].mean()
churn_by_tier    = df1.groupby('product_tier')['is_churned'].mean()

avg_onboarding_churned = df1[df1['is_churned']==1]['initial_onboarding_score'].mean()
avg_onboarding_retained = df1[df1['is_churned']==0]['initial_onboarding_score'].mean()

churn_by_channel = df1.groupby('acquisition_channel')['is_churned'].mean()
churn_by_segment_h2 = df1.groupby('sales_segment')['is_churned'].mean()

In [56]:
# Tenure in days (fill missing contract_end_dt with today for retained customers if needed)
today = pd.Timestamp.today()
df1['contract_end_dt_filled'] = df1['contract_end_dt'].fillna(today)
df1['tenure_days'] = (df1['contract_end_dt_filled'] - df1['contract_start_dt']).dt.days

# Discounted flag
df1['discounted'] = df1['discount_pct'] > 0

long_tenure_churn = df1[(df1['is_churned']==1) & (df1['tenure_days'] > 180)]

discounted_churn = df1[(df1['is_churned']==1) & (df1['discounted'])]

In [57]:
churn_by_segment

sales_segment
Enterprise    0.044304
MidMarket     0.045886
SMB_Field     0.168990
SMB_Inside    0.164394
Name: is_churned, dtype: float64

In [58]:
churn_by_size

company_size_bucket
1-10        0.264734
1000+       0.047904
11-50       0.046566
201-1000    0.042345
51-200      0.045886
Name: is_churned, dtype: float64

In [59]:
churn_by_tier

product_tier
Enterprise    0.055649
Growth        0.087295
Starter       0.194840
Name: is_churned, dtype: float64

In [60]:
avg_onboarding_churned

np.float64(5.871703296703298)

In [61]:
avg_onboarding_retained

np.float64(6.321775417298936)

# LOGO Per Quarter

In [62]:
df1["contract_end_dt"] = pd.to_datetime(df1["contract_end_date"], errors="coerce")

# Filter churned customers with valid end dates
churned = df1[(df1["is_churned"] == 1) & df1["contract_end_dt"].notna()]

# Create a quarter column
churned["churn_quarter"] = churned["contract_end_dt"].dt.to_period("Q")

churn_quarter_count = churned.groupby("churn_quarter")["customer_id"].nunique()
print(churn_quarter_count)

churn_quarter
2023Q2     30
2023Q3     71
2023Q4     71
2024Q1     87
2024Q2    105
Freq: Q-DEC, Name: customer_id, dtype: int64


/var/folders/_f/ttmxczgj72j0yw73z__5fnyw0000gn/T/ipykernel_36375/467477174.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  churned["churn_quarter"] = churned["contract_end_dt"].dt.to_period("Q")


# Churn Per Segment

In [63]:
q3_churn_by_segment = churned[churned["churn_quarter"] == "2023Q3"].groupby("sales_segment")["customer_id"].nunique()
print(q3_churn_by_segment)

sales_segment
Enterprise     4
MidMarket      7
SMB_Field     18
SMB_Inside    42
Name: customer_id, dtype: int64


In [64]:
q124_churn_by_segment = churned[churned["churn_quarter"] == "2024Q1"].groupby("sales_segment")["customer_id"].nunique()
q224_churn_by_segment = churned[churned["churn_quarter"] == "2024Q2"].groupby("sales_segment")["customer_id"].nunique()
q423_churn_by_segment = churned[churned["churn_quarter"] == "2023Q4"].groupby("sales_segment")["customer_id"].nunique()
q223_churn_by_segment = churned[churned["churn_quarter"] == "2023Q2"].groupby("sales_segment")["customer_id"].nunique()

q124_churn_by_segment

sales_segment
Enterprise     8
MidMarket      9
SMB_Field     19
SMB_Inside    51
Name: customer_id, dtype: int64

# Quarter 2 2024 Churn Spread

In [65]:
q224_churn_by_segment

sales_segment
Enterprise     8
MidMarket     11
SMB_Field     30
SMB_Inside    56
Name: customer_id, dtype: int64

# Quarter 2 2023 Churn Spread

In [66]:
q223_churn_by_segment

sales_segment
SMB_Field     12
SMB_Inside    18
Name: customer_id, dtype: int64

# Quarter 4 2023 Churn Spread

In [67]:
q423_churn_by_segment

sales_segment
Enterprise     1
MidMarket      2
SMB_Field     18
SMB_Inside    50
Name: customer_id, dtype: int64

# Customer Retention vs Churns

In [68]:
retained = df1[df1[CHURN_COL] == 0].copy()

# Total retained customers for percentage calculation
total_retained_logos = retained[ID_COL].nunique()
total_retained_acv = retained[ACV_COL].sum()

# 1️⃣ Retention by segment (can replace SEG_COL with 'company_size_bucket' or 'product_tier')
retention_summary = (
    retained.groupby(SEG_COL, dropna=False)
            .agg(retained_customers=(ID_COL, "nunique"),
                 retained_acv=(ACV_COL, "sum"))
            .reset_index()
)

# Add percentages relative to total retained
retention_summary["logo_retention_rate"] = retention_summary["retained_customers"] / retention_summary["retained_customers"].sum()
retention_summary["acv_retention_rate"] = retention_summary["retained_acv"] / retention_summary["retained_acv"].sum()

# Optional: merge with churn summary for side-by-side comparison
combined = summary[[SEG_COL, "churned_customers", "churned_acv", "logo_churn_share", "acv_churn_share"]].merge(
    retention_summary, on=SEG_COL, how="outer"
).fillna(0)

print("Retention by segment/size/tier:")
print(retention_summary.sort_values("retained_customers", ascending=False).to_string(index=False))

print("\nChurn vs Retention comparison:")
print(combined.sort_values("churned_customers", ascending=False).to_string(index=False))

Retention by segment/size/tier:
sales_segment  retained_customers  retained_acv  logo_retention_rate  acv_retention_rate
   SMB_Inside                1103    4840922.60             0.418437            0.047179
    MidMarket                 603   15127051.25             0.228756            0.147425
    SMB_Field                 477    2041748.28             0.180956            0.019898
   Enterprise                 453   80598576.22             0.171851            0.785498

Churn vs Retention comparison:
sales_segment  churned_customers  churned_acv  logo_churn_share  acv_churn_share  retained_customers  retained_acv  logo_retention_rate  acv_retention_rate
   SMB_Inside                217    595852.90          0.596154         0.114037                1103    4840922.60             0.418437            0.047179
    SMB_Field                 97    218822.49          0.266484         0.041879                 477    2041748.28             0.180956            0.019898
    MidMarket          

# How Many SMB did we pick up

In [69]:
df1["start_quarter"] = df1["contract_start_dt"].dt.to_period("Q")
df1["end_quarter"]   = df1["contract_end_dt"].dt.to_period("Q")
df1["end_quarter_filled"] = df1["end_quarter"].fillna("2025Q2")

In [70]:
# 1️⃣ Active customers per quarter by segment
quarters = pd.period_range(start=df1["start_quarter"].min(), end="2025Q2", freq="Q")

active_quarters = []
for q in quarters:
    active_in_q = df1[(df1["start_quarter"] <= q) & (df1["end_quarter_filled"] >= q)]
    counts = active_in_q.groupby("sales_segment")["customer_id"].nunique().reset_index()
    counts["quarter"] = q
    active_quarters.append(counts)

active_quarterly = pd.concat(active_quarters).sort_values(["quarter", "sales_segment"])
active_quarterly.rename(columns={"customer_id": "active_customers"}, inplace=True)

# 2️⃣ Churned customers per quarter by segment
# Only count churned accounts where contract ended in that quarter
churned_quarters = df1[df1["is_churned"] == 1].groupby(["end_quarter", "sales_segment"])["customer_id"].nunique().reset_index()
churned_quarters.rename(columns={"end_quarter": "quarter", "customer_id": "churned_customers"}, inplace=True)

# 3️⃣ New customers per quarter by segment
new_quarters = df1.groupby(["start_quarter", "sales_segment"])["customer_id"].nunique().reset_index()
new_quarters.rename(columns={"start_quarter": "quarter", "customer_id": "new_customers"}, inplace=True)

# Merge into a single dataframe
quarterly_summary = pd.merge(active_quarterly, churned_quarters, on=["quarter","sales_segment"], how="left")
quarterly_summary = pd.merge(quarterly_summary, new_quarters, on=["quarter","sales_segment"], how="left")

# Fill NaNs with 0 for churn/new counts
quarterly_summary["churned_customers"] = quarterly_summary["churned_customers"].fillna(0).astype(int)
quarterly_summary["new_customers"] = quarterly_summary["new_customers"].fillna(0).astype(int)

quarterly_summary.sort_values(["quarter","sales_segment"], inplace=True)
print(quarterly_summary)

   sales_segment  active_customers quarter  churned_customers  new_customers
0     Enterprise                87  2023Q1                  0             87
1      MidMarket               137  2023Q1                  0            137
2      SMB_Field               107  2023Q1                  0            107
3     SMB_Inside               238  2023Q1                  0            238
4     Enterprise               174  2023Q2                  0             87
5      MidMarket               257  2023Q2                  0            120
6      SMB_Field               227  2023Q2                 12            120
7     SMB_Inside               507  2023Q2                 18            269
8     Enterprise               277  2023Q3                  4            103
9      MidMarket               377  2023Q3                  7            120
10     SMB_Field               319  2023Q3                 18            104
11    SMB_Inside               776  2023Q3                 42            287